In [10]:
import torch
import torch.nn as nn
import transformers
import numpy as np
import umap
import plotly.graph_objects as go
import pandas as pd
import glob

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(torch.cuda.get_device_name(0))
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model = transformers.BertModel.from_pretrained('bert-base-uncased').to(device)

Using device: cuda
NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [11]:
# Define the path to the set file
set_file_path = r'C:\Users\franc\Desktop\teste\*.csv'
csv_files = glob.glob(set_file_path)
datasets = []
# Iterate through each CSV file and read it as a dataframe
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    datasets.append(df)

In [17]:
def plot_umap(datasets):

    # Definir parâmetros UMAP
    numero_vizinhos = 10
    distancia_minima = 0.1
    metrica = 'euclidean'

    # Definir cores para cada conjunto de dados
    cores = ['red', 'green', 'blue', 'purple', 'orange', 'brown', 'pink', 'cyan', 'gray', 'olive', 'yellow', 'black', 'teal', 'indigo', 'violet', 'wheat', 'navy', 'silver', 'maroon', 'lime', 'coral', 'darkblue', 'darkred']

    # Inicializar listas vazias para armazenar embeddings
    embeddings_cls = []
    embeddings_media_ponderada = []

    # Iterar através de cada conjunto de dados
    for indice, conjunto_dados in enumerate(datasets):
        # Tokenizar sequências de entrada e preencher sequências
        ids_entrada = [tokenizer.encode(texto, add_special_tokens=True, padding='max_length', max_length=128, truncation=True) for texto in conjunto_dados['final_smiles']]
        ids_entrada = torch.tensor(ids_entrada).to(device)

        # Criar máscaras de atenção
        mascara_atencao = [[float(i != 0.0) for i in seq] for seq in ids_entrada]
        mascara_atencao = torch.tensor(mascara_atencao).to(device)

        # Reduzir o tamanho do lote se necessário
        tamanho_lote = 20  # ou qualquer número que seja adequado para sua GPU
        for inicio in range(0, len(ids_entrada), tamanho_lote):
            fim = inicio + tamanho_lote
            lote_ids_entrada = ids_entrada[inicio:fim]
            lote_mascara_atencao = mascara_atencao[inicio:fim]

            with torch.no_grad():
                saidas = model(lote_ids_entrada, attention_mask=lote_mascara_atencao)
            
                # Obter embeddings para todos os tokens, não apenas o token CLS
                embeddings = saidas.last_hidden_state
            
                # Pooling médio
                somatorio_embeddings = torch.sum(embeddings * lote_mascara_atencao.unsqueeze(-1), 1)
                embeddings_media_ponderada.append(somatorio_embeddings / lote_mascara_atencao.sum(1).unsqueeze(-1))

    # Aplicar UMAP
    umap_cls = umap.UMAP(n_neighbors=numero_vizinhos, min_dist=distancia_minima, metric=metrica).fit_transform(embeddings_cls)
    umap_media_ponderada = umap.UMAP(n_neighbors=numero_vizinhos, min_dist=distancia_minima, metric=metrica).fit_transform(embeddings_media_ponderada)

    # Plotar visualização UMAP
    figura = go.Figure()

    # Adicionar rastros à figura para cada conjunto de dados
    for indice, conjunto_dados in enumerate(datasets):
        figura.add_trace(go.Scatter(x=umap_cls[indice, 0], y=umap_cls[indice, 1], mode='markers', name=conjunto_dados.file_name))

plot_umap(datasets)

KeyboardInterrupt: 